# Automated Question-Answer Generation from Pharmaceutical Data

This notebook systematically generates question-answer pairs from pharmaceutical data sources including:
1. PubMed target passages
2. DrugBank tables
3. Related pharmaceutical data

Each QA pair will include:
- Question
- Answer
- Source text/passage
- Related table

In [1]:
# Import required libraries
import os
import pandas as pd
import json
from pathlib import Path
from openai import OpenAI
import csv
from tqdm import tqdm
from collections import defaultdict

In [8]:
# Configuration and paths
PUBMED_TARGETS_DIR = '../data/Pharma/pubmed-targets'
DRUGBANK_TABLES_DIR = '../data/Pharma/drugbank-tables'
MAPPING_FILE = '../data/Pharma/pubmed-drugbank-tables.gt'
OUTPUT_FILE = 'table_output.gt'

# Initialize your LLM API key if needed
# os.environ["OPENAI_API_KEY"]="<KEY>"
client = OpenAI()


In [9]:
def load_passage_table_mapping():
    """Load the mapping between passages and their relevant tables"""
    mapping = defaultdict(list)
    with open(MAPPING_FILE, 'r') as f:
        for line in f:
            passage_id, table_name = line.strip().split(',')
            mapping[passage_id].append(table_name)
    # print("mapping", mapping)
    return mapping

def load_target_passages():
    """Load all target passages from the pubmed-targets directory"""
    passages = {}
    target_files = Path(PUBMED_TARGETS_DIR).glob('Target-*')
    
    for file_path in target_files:
        target_id = file_path.name
        with open(file_path, 'r') as f:
            passages[target_id] = f.read()
    # print("passages", passages)
    return passages

def load_drugbank_tables():
    """Load all relevant DrugBank tables"""
    tables = {}
    csv_files = Path(DRUGBANK_TABLES_DIR).glob('*.csv')
    
    for file_path in csv_files:
        table_name = file_path.stem
        tables[table_name] = pd.read_csv(file_path)
    
    # print("tables", tables)
    return tables

In [36]:
def get_relevant_table_content(table_id, table_content, max_rows=5):
    # """Extract relevant content from tables for context"""
    # print("Debug - Available tables:", tables.keys())
    # print("Debug - Looking for table_names:", table_names)

    total_max_rows = 500
    max_rows_per_table = 500
    table_content = {}
    remaining_rows = total_max_rows
    valid_tables = [name.replace('.csv', '') for name in table_names if name.replace('.csv', '') in tables]
    
    # Calculate rows per table based on available tables
    if valid_tables:
        rows_per_table = min(max_rows_per_table, remaining_rows // len(valid_tables))
    else:
        rows_per_table = 0

    table_content = {}
    for table_name in table_names:
        # Remove .csv extension if present
        base_table_name = table_name.replace('.csv', '')
        
        if base_table_name in tables:
            df = tables[base_table_name]
            
            # Take fewer rows for large tables
            rows_to_take = min(rows_per_table, len(df))
            
            table_content[base_table_name] = {
                'columns': list(df.columns),
                'data': df.head(rows_to_take).to_dict('records'),
                'total_rows': len(df)  # Include total row count for reference
            }
            
            remaining_rows -= rows_to_take
        else:
            print(f"Debug - Table '{base_table_name}' not found in available tables")
    return table_content

def generate_questions_for_table(table_id, table_content, model="gpt-4o"):
    """Generate questions for a given passage and its relevant tables using LLM"""

    # print("tables", tables)
    # print("relevant_table_names", relevant_table_names)
    
    # Limit passage length if too long (e.g., first 1000 characters)
    # passage_text = passage_text[:1000] + "..." if len(passage_text) > 1000 else passage_text
    
    # # Limit to maximum 3 relevant tables
    # relevant_table_names = relevant_table_names[:1]
    
    # # Get relevant table content
    # table_content = get_relevant_table_content(table_id, table_content)
    
    df = table_content
            
    # Instead of fixed row count, adaptively limit content based on estimated token size
    max_tokens = 10000  # Conservative estimate of available tokens for table content
    
    # Function to estimate tokens in a row (rough approximation: ~4 chars per token)
    def estimate_row_tokens(row):
        row_str = str(row.to_dict())
        return len(row_str) // 4
    
    # Sample a few rows to estimate average row size
    sample_size = min(5, len(df))
    if sample_size > 0:
        sample_rows = df.sample(sample_size) if len(df) > 5 else df
        avg_tokens_per_row = sum(estimate_row_tokens(row) for _, row in sample_rows.iterrows()) // sample_size
        
        # Calculate how many rows we can include
        rows_to_take = min(max_tokens // max(1, avg_tokens_per_row), len(df))
        
        # Take at least 5 rows but no more than 100 (as a reasonable default)
        rows_to_take = max(5, min(100, rows_to_take))
    else:
        rows_to_take = 0
    
    formatted_table_content = {
        'table_name': table_id,
        'columns': list(df.columns),
        'data': df.head(rows_to_take).to_dict('records'),
        'total_rows': len(df)  # Include total row count for reference
    }
    
    prompt = f"""
    Given the following passage and related tables, generate 2 meaningful question-answer pairs.
    IMPORTANT: Each question MUST be answerable using information from ONLY the tables.
    Only generate questions that require information from the tables. Try and make the question as difficult and technical as possible.
    
    You are a pharmaceutical educator creating realistic questions that a medical student, pharmacist, or healthcare professional might ask. Focus on clinically relevant information such as:
    - Drug mechanisms, indications, and therapeutic uses
    - Dosing considerations and administration routes
    - Side effects and contraindications
    - Drug interactions and pharmacological properties
    
    AVOID generating questions about:
    - Database-specific information (primary keys, IDs, URLs)
    - Technical metadata that wouldn't be useful in clinical practice
    - Questions that simply ask to list information without clinical context
    
    Examples of BAD questions to AVOID:
    - "What drug is linked to the FDA label with the URL ending in '1265922800,' and what is its state?"
    - "What is the primary key for the drug Denileukin diftitox, and what is its marketed mixture name?"
    
    Tables:
    {json.dumps(formatted_table_content, indent=2)}
    
    Generate questions in the following format:
    1. question: [specific question about drug/treatment that would be asked in clinical practice]
       answer: [detailed answer combining information from the tables]
       text: None
       table: [table name(s) if information from table was used (e.g. drugbank-drug)]
    
    Ensure every question is clinically relevant and would be asked by real healthcare professionals.
    """
    
    # Call your LLM here with the prompt
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical and pharmaceutical expert tasked with generating detailed question-answer pairs about drugs and treatments."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    print(response.choices[0].message.content)
    
    # Parse the response into structured QA pairs
    qa_pairs = parse_llm_response(response.choices[0].message.content)
    return qa_pairs

def parse_llm_response(response_text):
    """Parse the LLM response into structured QA pairs"""
    qa_pairs = []
    
    # Split the response into individual QA entries
    entries = response_text.strip().split('\n\n')
    
    for entry in entries:
        if not entry.strip():
            continue
            
        lines = entry.strip().split('\n')
        current_qa = {
            'question': '',
            'answer': '',
            'text': 'None',
            'table': ''  # Default value
        }
        
        for line in lines:
            line = line.strip()
            # Skip empty lines and numbering
            if not line or line.replace('.', '').strip().isdigit():
                continue
                
            # Parse each field using more robust splitting
            if 'question:' in line:
                current_qa['question'] = line.split('question:', 1)[1].strip()
            elif 'answer:' in line:
                current_qa['answer'] = line.split('answer:', 1)[1].strip()
            elif 'text:' in line:
                current_qa['text'] = line.split('text:', 1)[1].strip()
            elif 'table:' in line:
                table_value = line.split('table:', 1)[1].strip()
                # Handle NA, N/A, None cases
                current_qa['table'] = 'None' if table_value.upper() in ['NA', 'N/A', 'NONE'] else table_value
        
        # Only add complete QA pairs that have both question and answer
        if current_qa['question'] and current_qa['answer']:
            qa_pairs.append(current_qa.copy())  # Use copy to avoid reference issues
    
    return qa_pairs

In [37]:
def main():
    # Load mappings and data
    print("Loading passage-table mappings...")
    passage_table_mapping = load_passage_table_mapping()
    
    print("Loading target passages...")
    passages = load_target_passages()
    
    print("Loading DrugBank tables...")
    tables = load_drugbank_tables()
    
    # Initialize output list
    qa_pairs = []
    
    # Process each passage with its relevant tables
    for table_id, table_content in tqdm(list(tables.items())[:20]):
        
        # print(passage_id, passage_text, tables, relevant_tables)
        
        # Generate QA pairs using the passage and its relevant tables
        new_qa_pairs = generate_questions_for_table(
            table_id,
            table_content
        )
        # new_qa_pairs = []
        print(new_qa_pairs)
        qa_pairs.extend(new_qa_pairs)
    
    # Save results
    with open(OUTPUT_FILE, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for qa_pair in qa_pairs:
            writer.writerow([
                qa_pair['question'],
                qa_pair['answer'],
                qa_pair['text'],
                qa_pair['table']
            ])
        
    print(f"Generated {len(qa_pairs)} question-answer pairs")

In [38]:
if __name__ == "__main__":
    main()

Loading passage-table mappings...
Loading target passages...
Loading DrugBank tables...


  5%|▌         | 1/20 [00:06<02:03,  6.48s/it]

1. question: How might a healthcare professional verify the involvement of the EGFR gene in a therapeutic target using multiple scientific resources?
   answer: A healthcare professional can verify the involvement of the EGFR gene in a therapeutic target by consulting several scientific resources. According to the table "drugbank-targets_polypeptides_ext_id," the EGFR gene is associated with the parent key "BE0000767." This gene can be cross-referenced through various resources such as the HUGO Gene Nomenclature Committee (HGNC) with the identifier "HGNC:3236," GenAtlas with the identifier "EGFR," GenBank Gene Database with the identifier "X00588," GenBank Protein Database with the identifier "757924," Guide to Pharmacology with the identifier "1797," UniProtKB with the identifier "P00533," and UniProt Accession with the identifier "EGFR_HUMAN."
   text: None
   table: drugbank-targets_polypeptides_ext_id

2. question: In the context of targeting the immune system, how could a healthca

 10%|█         | 2/20 [00:10<01:31,  5.06s/it]

1. question: What is the cost difference between a single dose of Neulasta and a single dose of Pegasys when administered in their typical syringe and vial forms, respectively?
   answer: The cost of Neulasta 6 mg/0.6 ml syringe is $4102.37, while the cost of Pegasys 180 mcg/ml vial is $642.64. The cost difference between a single dose of Neulasta and Pegasys, when administered in their typical forms, is $4102.37 - $642.64 = $3459.73.
   text: None
   table: drugbank-drug_prices

2. question: If a physician prescribes a 10-day treatment using the most expensive form of Aranesp available in the database, what would be the total cost?
   answer: The most expensive form of Aranesp is the "Aranesp (Albumin Free) 150 mcg/0.75ml Solution (1 Box = Four 0.75ml Vials)" which costs $3902.75 per box. Assuming a 10-day treatment requires daily administration and each box contains four 0.75ml vials (sufficient for four days), the total cost for a 10-day treatment would be 3 boxes (covering 12 days)

...

1. question: What are the different synonyms for the protein that acts as a carrier called "Brain lipid-binding protein," and what could be its potential role concerning drug binding or transport mechanisms?
   answer: The protein known as "Brain lipid-binding protein" is synonymously referred to as B-FABP, BLBP, Brain-type fatty acid-binding protein, FABPB, Fatty acid-binding protein 7, Mammary-derived growth inhibitor related, and MRG. As a lipid-binding protein, it could potentially play a role in the transport or binding of drugs that are lipid-soluble, affecting their distribution within the brain.
   text: None
   table: drugbank-carriers_polypeptides_syn

2. question: How does the protein known as "Serpin A6" relate to corticosteroid transport, and what are its synonyms that might be used in literature or clinical discussions?
   answer: The protein "Serpin A6" is involved in corticosteroid transport as it is synonymously known as CBG (corticosteroid-binding globulin) and Transc

 95%|█████████▌| 19/20 [01:29<00:04,  4.51s/it]

1. question: Which drug in the database is both approved and has been withdrawn, and what might this imply about its clinical use?
   answer: The drug with DrugBank ID DB00010 is both approved and withdrawn. This dual status might imply that, although the drug was once approved for clinical use, it was later withdrawn due to safety concerns, adverse effects, or the availability of better alternatives.
   text: None
   table: drugbank-drug_groups

2. question: Identify a drug in the database that is approved for use but also has a status indicating it is being investigated. Why might a drug have such dual classifications?
   answer: The drug with DrugBank ID DB00004 is both approved and investigational. A drug can have such dual classifications if it is approved for certain indications but is still under investigation for additional uses or in combination with other therapies, aiming to expand its clinical applications or improve its efficacy and safety profile.
   text: None
   table: 

100%|██████████| 20/20 [01:34<00:00,  4.73s/it]

1. question: What is the therapeutic application of leuprolide acetate, as indicated by the FDA labels for its various formulations, and what makes it unique compared to other drug therapies?
   answer: Leuprolide acetate is utilized in various formulations such as Lupron Depot, Lupron Depot-Ped, Fensolvi, and Eligard, which are all depot suspensions, indicating its role in long-term hormonal therapy. These formulations are used primarily for conditions like prostate cancer, endometriosis, central precocious puberty, and uterine fibroids, as noted by the FDA labels. Its uniqueness lies in its sustained-release formulation, which allows for less frequent dosing and better patient compliance in chronic conditions requiring hormonal suppression.
   text: None
   table: drugbank-drug_targ_links

2. question: How does the administration route and formulation of octreotide differentiate it from other somatostatin analogs in terms of clinical use according to the FDA and NIH resources?
   ans